연관된 package 로딩하기.

In [1]:
import argparse
import torch
import torch.nn as nn
from torchvision import datasets, transforms

# 모델생성하기

In [3]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5, 1) # 6 * 24 24 
        self.pool1 = nn.MaxPool2d(2)        # 6 * 12, 12
        self.conv2 = nn.Conv2d(6, 16, 5, 1) # 16 * 8 * 8
        self.pool2 = nn.MaxPool2d(2)        #  16 * 4 * 4 
        self.conv_module = nn.Sequential(
            self.conv1, nn.ReLU(), self.pool1, 
            self.conv2, nn.ReLU(), self.pool2
        )
        self.fc1 = nn.Linear( 16 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, 84)
        self.fc3 = nn.Linear(84, 10)
        self.fc_module = nn.Sequential(
            self.fc1, nn.ReLU(), self.fc2, nn.ReLU(), self.fc3
        )       
        

    def forward(self, x):
        x = self.conv_module(x)
        dim = 1
        for d in x.size()[1:]: #16, 4, 4
            dim = dim * d
        x = x.view(-1, dim)    
        x = self.fc_module(x)

        return x 

# 메인 함수 만들기

In [4]:
def main():
    parser = argparse.ArgumentParser()
    args = parser.parse_args("")
    args.batch_size = 128
    args.test_batch_size = 128    
    args.epochs = 2
    args.lr = 0.001

    list_train_loss = []
    list_val_loss = []
    list_epochs = []
    
    train_dataset = datasets.MNIST("../data", train=True, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])  , download=True)
    train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [50000, 10000])                    
    test_dataset = datasets.MNIST("../data", train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ]), download=True)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=128, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=True)
    
    
    model = CNNModel()
    print('Number of {} parameters'.format(sum(p.numel() for p in model.parameters() if p.requires_grad)))
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam( model.parameters(), lr= args.lr)
    #criterion = nn.MSELoss
    
    # training 작업진행.
    
    for ind in range(args.epochs):
        # Training
        train_loss = 0
        model.train()
        for data, target in train_loader:                                    
            preds = model( data )            
            loss = criterion( preds, target )            
            train_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        train_loss = train_loss / len( train_loader)    
        print("train_loss", train_loss )    
        list_train_loss.append( train_loss )
        list_epochs.append(ind)

        #Validation
        val_loss = 0
        model.eval()
        for data, target in val_loader:                              
            preds = model( data )            
            loss = criterion( preds, target )            
            val_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()            
        val_loss = val_loss / len( val_loader)    
        print("val_loss", val_loss )    
        list_val_loss.append( train_loss )

        # Test
        correct = 0
        model.eval()        
        optimizer.zero_grad()

        for data , target in test_loader:
            #data = data.view(-1, 28 * 28)
            preds = model( data )        
            preds_y = preds.max(1, keepdim=True)[1].squeeze()        
            local_correct_sum = preds_y.eq(target).sum()
            correct += local_correct_sum            
        acc = correct.numpy() / len( test_loader.dataset )    
        print("Epoch: {}, Train Loss: {}, Val Loss: {}, Test Acc: {}%".format(ind, train_loss, 0, acc * 100))        


    print(" training done..")

In [ ]:
실제 실행하기